<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/RL/6-2_dqn_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym
!pip install torch
!pip install numpy
!pip install random
!pip install collections
!pip install torch.optim
!pip install torch.nn

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 5.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch.nn (from versions: none)
ERROR: No matching distribution found for torch.nn


In [2]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

# Q-Network 모델 정의
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Hyperparameters
state_size = 4
action_size = 2
batch_size = 64
gamma = 0.99  # 할인율
epsilon = 1.0  # 탐험률
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
target_update = 10

# Replay Memory
memory = deque(maxlen=2000)

# 모델과 타겟 네트워크 초기화
q_network = QNetwork(state_size, action_size)
target_network = QNetwork(state_size, action_size)
target_network.load_state_dict(q_network.state_dict())  # 가중치 복사
optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)

# 경험 샘플링 함수
def replay(memory, batch_size):
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*minibatch)

    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.long)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    # 현재 상태에서의 Q값 계산
    q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)

    # 다음 상태에서의 Q값 계산 (타겟 네트워크 사용)
    next_q_values = target_network(next_states).max(1)[0]
    expected_q_values = rewards + (gamma * next_q_values * (1 - dones))

    # 손실 계산 및 역전파
    loss = nn.MSELoss()(q_values, expected_q_values.detach())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 행동 선택 함수 (ε-greedy)
def choose_action(state, epsilon):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        q_values = q_network(state)
    return np.argmax(q_values.numpy())

# CartPole 환경 설정
env = gym.make('CartPole-v1')
episodes = 1000

# 학습 루프
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        # 행동 선택
        action = choose_action(state, epsilon)

        # 환경에서 한 단계 진행
        next_state, reward, done, _ = env.step(action)

        # 보상 조정
        reward = reward if not done else -10

        # 메모리에 저장
        memory.append((state, action, reward, next_state, done))

        # 상태 업데이트
        state = next_state
        total_reward += reward

        # 경험 리플레이
        replay(memory, batch_size)

    # 탐험률 감소
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # 타겟 네트워크 업데이트
    if episode % target_update == 0:
        target_network.load_state_dict(q_network.state_dict())

    print(f"Episode: {episode}, Total reward: {total_reward}, Epsilon: {epsilon}")

# 학습 완료 후 에이전트 테스트
for i in range(10):
    state = env.reset()
    done = False
    while not done:
        env.render()
        action = choose_action(state, epsilon_min)  # 최종 탐험률로 행동 선택
        next_state, _, done, _ = env.step(action)
        state = next_state
env.close()


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
<ipython-input-2-4a33df966720>:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarra

Episode: 0, Total reward: 23.0, Epsilon: 0.995
Episode: 1, Total reward: 24.0, Epsilon: 0.990025
Episode: 2, Total reward: 12.0, Epsilon: 0.985074875
Episode: 3, Total reward: 10.0, Epsilon: 0.9801495006250001
Episode: 4, Total reward: 15.0, Epsilon: 0.9752487531218751
Episode: 5, Total reward: 13.0, Epsilon: 0.9703725093562657
Episode: 6, Total reward: 18.0, Epsilon: 0.9655206468094844
Episode: 7, Total reward: 4.0, Epsilon: 0.960693043575437
Episode: 8, Total reward: 87.0, Epsilon: 0.9558895783575597
Episode: 9, Total reward: 5.0, Epsilon: 0.9511101304657719
Episode: 10, Total reward: 24.0, Epsilon: 0.946354579813443
Episode: 11, Total reward: 13.0, Epsilon: 0.9416228069143757
Episode: 12, Total reward: 25.0, Epsilon: 0.9369146928798039
Episode: 13, Total reward: 22.0, Epsilon: 0.9322301194154049
Episode: 14, Total reward: 5.0, Epsilon: 0.9275689688183278
Episode: 15, Total reward: 6.0, Epsilon: 0.9229311239742362
Episode: 16, Total reward: 13.0, Epsilon: 0.918316468354365
Episode: 1

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
